In [1]:
import pandas as pd 
pd.set_option('display.max_colwidth', None)
import numpy as np

In [2]:
import wandb
from IPython.display import display

In [3]:
api = wandb.Api()
entity, project = "gmum", "conditional_contrastive"  # set to your entity and project 
runs = api.runs(entity + "/" + project, ) 
len(runs)

99

In [4]:
runs = [r for r in runs if r.name.startswith("moco-resnet50-imagenet100") ]

len(runs)

7

In [5]:
linear_sets = {
    k
    for r in runs
    for k in r.summary.keys() if k.startswith("test_")
    if "stl" not in k
}

linear_sets

{'test_few-shot/fc100',
 'test_few-shot_5-way_1-shot/cub200',
 'test_few-shot_5-way_1-shot/fc100',
 'test_few-shot_5-way_1-shot/plant_disease',
 'test_few-shot_5-way_1-way/fc100',
 'test_few-shot_5-way_5-shot/cub200',
 'test_few-shot_5-way_5-shot/fc100',
 'test_few-shot_5-way_5-shot/plant_disease',
 'test_few_shot/cub200',
 'test_few_shot/fc100',
 'test_few_shot/plant_disease',
 'test_linear/aircraft',
 'test_linear/caltech101',
 'test_linear/cars',
 'test_linear/cifar10',
 'test_linear/cifar100',
 'test_linear/dtd',
 'test_linear/flowers',
 'test_linear/food101',
 'test_linear/mit67',
 'test_linear/pets',
 'test_linear/sun397',
 'test_linear_looc_like_acc/cub200',
 'test_linear_looc_like_acc/imagenet100',
 'test_linear_looc_like_best_acc/cub200',
 'test_linear_looc_like_best_acc/imagenet100',
 'test_linear_looc_like_v2_acc/cub200',
 'test_linear_looc_like_v2_best_acc/cub200'}

In [6]:
runs[-1].summary["test_linear/food101"]

0.6123168468475342

In [7]:
paper_stl_rows = [
    {
        "name": "moco-resnet18-stl10_BASE_PAPER",
        'test_linear/aircraft': 0.2663,
        'test_linear/caltech101': 0.6415,
        'test_linear/cars': 0.1609,
        'test_linear/cifar10': 0.818,
        'test_linear/cifar100': 0.5375,
        'test_linear/dtd': 0.412,
        'test_linear/flowers': 0.6101,
        'test_linear/food101': 0.3369,
        'test_linear/mit67': 0.3901,
        'test_linear/pets': 0.4234,
        'test_linear/sun397': 0.285,
    },
    {
        "name": "moco-resnet18-stl10_AUGSELF",
        'test_linear/aircraft': 0.2802,
        'test_linear/caltech101': 0.6602,
        'test_linear/cars': 0.1753,
        'test_linear/cifar10': 0.8245,
        'test_linear/cifar100': 0.5717,
        'test_linear/dtd': 0.4521,
        'test_linear/flowers': 0.6696,
        'test_linear/food101': 0.3691,
        'test_linear/mit67': 0.4167,
        'test_linear/pets': 0.4380,
        'test_linear/sun397': 0.3093,
    }

]

paper_imagenet_rows = [
    {
        "name": "moco-resnet50-imagenet100_BASELINE_PAPER",
        'test_linear/aircraft': 0.4121,
        'test_linear/caltech101': 0.7725,
        'test_linear/cars': 0.3386,
        'test_linear/cifar10': 0.846,
        'test_linear/cifar100': 0.616,
        'test_linear/dtd': 0.6447,
        'test_linear/flowers': 0.8243,
        'test_linear/food101': 0.5967,
        'test_linear/mit67': 0.6164,
        'test_linear/pets': 0.7008,
        'test_linear/sun397': 0.4650,
        
        "test_few-shot_5-way_1-shot/fc100": 0.3167,
        "test_few-shot_5-way_5-shot/fc100": 0.4388,
        
        "test_few-shot_5-way_1-shot/cub200": 0.4167,
        "test_few-shot_5-way_5-shot/cub200": 0.5692,
        
        "test_few-shot_5-way_1-shot/plant_disease": 0.6573,
        "test_few-shot_5-way_5-shot/plant_disease": 0.8498,
        
        "test_linear_looc_like_v2_best_acc/cub200": 32.2,
    },
    {
        "name": "moco-resnet50-imagenet100_AUGSELF_PAPER",
        'test_linear/aircraft': 0.3947,
        'test_linear/caltech101': 0.7893,
        'test_linear/cars':0.3735,
        'test_linear/cifar10': 0.8526,
        'test_linear/cifar100': 0.639,
        'test_linear/dtd': 0.6622,
        'test_linear/flowers': 0.8570,
        'test_linear/food101': 0.6078,
        'test_linear/mit67': 0.6336,
        'test_linear/pets': 0.7346,
        'test_linear/sun397': 0.4852,
        
        "test_few-shot_5-way_1-shot/fc100": 0.3502,
        "test_few-shot_5-way_5-shot/fc100": 0.4877,
        
        "test_few-shot_5-way_1-shot/cub200": 0.4417,
        "test_few-shot_5-way_5-shot/cub200": 0.5735,
        
        "test_few-shot_5-way_1-shot/plant_disease": 0.718,
        "test_few-shot_5-way_5-shot/plant_disease": 0.8781,
        
        "test_linear_looc_like_v2_best_acc/cub200": 37.0,
    },
    
    {
        "name": "moco-resnet50-imagenet100_LOOC_PAPER",
        "test_linear_looc_like_v2_best_acc/cub200": 40.1,

    }
    
]

In [8]:
summary_list, config_list, name_list = [], [], []

rows = [] + paper_imagenet_rows #+ paper_rows

for run in runs:
    
    rows.append({
        "name": run.name,
        "run": run,
        **{
            k: run.summary.get(k, 0)
            for k in linear_sets
        }
    
    })

runs_df = pd.DataFrame(rows)

In [9]:
runs_df

,name,test_linear/aircraft,test_linear/caltech101,test_linear/cars,test_linear/cifar10,test_linear/cifar100,test_linear/dtd,test_linear/flowers,test_linear/food101,test_linear/mit67,...,test_few_shot/fc100,test_few-shot/fc100,test_linear_looc_like_v2_acc/cub200,test_few-shot_5-way_1-way/fc100,test_few_shot/plant_disease,test_linear_looc_like_best_acc/cub200,test_few_shot/cub200,test_linear_looc_like_acc/cub200,test_linear_looc_like_best_acc/imagenet100,test_linear_looc_like_acc/imagenet100
0,moco-resnet50-imagenet100_BASELINE_PAPER,0.412100,0.772500,0.338600,0.8460,0.6160,0.644700,0.824300,0.596700,0.616400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,moco-resnet50-imagenet100_AUGSELF_PAPER,0.394700,0.789300,0.373500,0.8526,0.6390,0.662200,0.857000,0.607800,0.633600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,moco-resnet50-imagenet100_LOOC_PAPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,moco-resnet50-imagenet100_mlp_3_256_proj-cat_crop_color,0.428467,0.815819,0.391867,0.8654,0.6619,0.666489,0.857597,0.615089,0.650746,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,moco-resnet50-imagenet100_baseline_OFFICIAL,0.410036,0.772417,0.338142,0.8458,0.6159,0.644149,0.823412,0.595406,0.615672,...,0.457837,0.324647,32.050396,0.000000,0.852275,0.000000,0.000000,0.000000,0.000000,0.000000
5,moco-resnet50-imagenet100_augself_OFFICIAL,0.394688,0.789258,0.376446,0.8508,0.6363,0.664894,0.858014,0.610218,0.632836,...,0.489706,0.484731,33.690025,0.348333,0.883937,29.513288,0.571694,29.219883,82.879997,82.599998
6,moco-resnet50-imagenet100_mlp_3_16_proj-cat_crop_color,0.421551,0.819286,0.394603,0.8628,0.6540,0.677128,0.855731,0.611842,0.641045,...,0.490419,0.000000,33.293060,0.000000,0.000000,33.396618,0.569381,33.224026,0.000000,0.000000
7,moco-resnet50-imagenet100_augself,0.426292,0.812339,0.376073,0.8587,0.6534,0.000000,0.871275,0.622297,0.651493,...,0.000000,0.000000,29.668623,0.000000,0.000000,24.905073,0.000000,24.663445,84.659996,84.459999
8,moco-resnet50-imagenet100_mlp_2_32_proj-cat_crop_color,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,moco-resnet50-imagenet100_mlp_2_16_proj-cat_crop_color,0.420882,0.821014,0.387390,0.8648,0.6444,0.664894,0.859430,0.612317,0.648507,...,0.502925,0.000000,32.775284,0.000000,0.873675,33.016914,0.570188,32.637211,81.519997,80.900002


In [10]:
# df.style.highlight_max()

In [11]:


for m_subset in [
    "test_linear/",
    "test_few-shot_5-way",
    "test_linear_looc_like_v2_best_acc/"
]:
    df = runs_df[["name"] + [c for c in runs_df.columns if c.startswith(m_subset) and "way/" not in c and not c.endswith("rank")]].copy().dropna()
    df["name"] = df.name.map(
        lambda n: n.replace("moco-resnet50-imagenet100_", "").replace("OFFICIAL", "OFFICIAL_WEIGHTS").upper()
    )
    
    for n in [
        # "MLP_3_256_PROJ-CAT_CROP_COLOR",
        "MLP_2_32_PROJ-CAT_CROP_COLOR"
    ]:
        df = df[df.name!=n]
    
    df["name"] = df.name.map(
        lambda n: n if any([n.startswith(p) for p in ["AUGSELF", "LOOC", "BASELINE"] ]) else f"OUR_{n}"
    )
    df.style.highlight_max(color = 'blue', axis = 0)
    
    dn = len(df)
    c_subset = [c for c in df.columns if c!="name"]
    print("""
GREEN - 1
YELLOW - 2
PINK - 3
""")
    display(
        df.sort_values("name").style.
            highlight_quantile(
                props="font-style: normal; background: pink",
                q_left=(dn-3)/dn, q_right=(dn-2)/dn,
                subset=c_subset,
            ).
            highlight_quantile(
                props="font-style: italic; background: yellow",
                q_left=(dn-2)/dn, q_right=(dn-1)/dn,
                subset=c_subset,
            ).
            highlight_max(
                props="font-weight: bold; background: lightgreen",
                subset=c_subset,
            )
    )
    # print(df[df.columns[1]])


GREEN - 1
YELLOW - 2
PINK - 3



,name,test_linear/aircraft,test_linear/caltech101,test_linear/cars,test_linear/cifar10,test_linear/cifar100,test_linear/dtd,test_linear/flowers,test_linear/food101,test_linear/mit67,test_linear/pets,test_linear/sun397
7,AUGSELF,0.426292,0.812339,0.376073,0.858700,0.653400,0.000000,0.871275,0.622297,0.651493,0.758842,0.500705
5,AUGSELF_OFFICIAL_WEIGHTS,0.394688,0.789258,0.376446,0.850800,0.636300,0.664894,0.858014,0.610218,0.632836,0.734562,0.485239
1,AUGSELF_PAPER,0.394700,0.789300,0.373500,0.852600,0.639000,0.662200,0.857000,0.607800,0.633600,0.734600,0.485200
4,BASELINE_OFFICIAL_WEIGHTS,0.410036,0.772417,0.338142,0.845800,0.615900,0.644149,0.823412,0.595406,0.615672,0.699543,0.468212
0,BASELINE_PAPER,0.412100,0.772500,0.338600,0.846000,0.616000,0.644700,0.824300,0.596700,0.616400,0.700800,0.465000
9,OUR_MLP_2_16_PROJ-CAT_CROP_COLOR,0.420882,0.821014,0.387390,0.864800,0.644400,0.664894,0.859430,0.612317,0.648507,0.743864,0.497229
6,OUR_MLP_3_16_PROJ-CAT_CROP_COLOR,0.421551,0.819286,0.394603,0.862800,0.654000,0.677128,0.855731,0.611842,0.641045,0.751116,0.494207
3,OUR_MLP_3_256_PROJ-CAT_CROP_COLOR,0.428467,0.815819,0.391867,0.865400,0.661900,0.666489,0.857597,0.615089,0.650746,0.735193,0.000000



GREEN - 1
YELLOW - 2
PINK - 3



,name,test_few-shot_5-way_1-shot/fc100,test_few-shot_5-way_5-shot/fc100,test_few-shot_5-way_1-shot/cub200,test_few-shot_5-way_5-shot/cub200,test_few-shot_5-way_1-shot/plant_disease,test_few-shot_5-way_5-shot/plant_disease
7,AUGSELF,0.380225,0.514888,0.442712,0.588081,0.738631,0.906494
5,AUGSELF_OFFICIAL_WEIGHTS,0.349381,0.489706,0.445425,0.571694,0.724350,0.883937
1,AUGSELF_PAPER,0.350200,0.487700,0.441700,0.573500,0.718000,0.878100
4,BASELINE_OFFICIAL_WEIGHTS,0.321813,0.457837,0.421777,0.000000,0.659612,0.852275
0,BASELINE_PAPER,0.316700,0.438800,0.416700,0.569200,0.657300,0.849800
9,OUR_MLP_2_16_PROJ-CAT_CROP_COLOR,0.356931,0.502925,0.437006,0.570188,0.711969,0.873675
6,OUR_MLP_3_16_PROJ-CAT_CROP_COLOR,0.356037,0.490419,0.438744,0.569381,0.716825,0.875462
3,OUR_MLP_3_256_PROJ-CAT_CROP_COLOR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000



GREEN - 1
YELLOW - 2
PINK - 3



,name,test_linear_looc_like_v2_best_acc/cub200
7,AUGSELF,29.892992
5,AUGSELF_OFFICIAL_WEIGHTS,33.793579
1,AUGSELF_PAPER,37.000000
4,BASELINE_OFFICIAL_WEIGHTS,32.171211
0,BASELINE_PAPER,32.200000
2,LOOC_PAPER,40.100000
9,OUR_MLP_2_16_PROJ-CAT_CROP_COLOR,32.947876
6,OUR_MLP_3_16_PROJ-CAT_CROP_COLOR,33.431137
3,OUR_MLP_3_256_PROJ-CAT_CROP_COLOR,0.000000


In [12]:
# df.style

In [38]:
perf_index = list(range(len(runs_df)))


for k in linear_sets:
    subdf = runs_df[["name", k]].sort_values(k, ascending=False)
    
    subdf["rank"] = perf_index
    
    base_paper_index = subdf.index.get_loc(0)

    print("-----", k.upper(), "------")
    
    display(
        subdf.head(base_paper_index+2)
    )
    
    
    runs_df[f"{k}/rank"] = subdf["rank"]
    
    
    
runs_df["rank_mean"] = runs_df.apply(
    lambda r: np.mean([r[f"{c}/rank"] for c in linear_sets]),
    axis=1
) 

runs_df["rank_std"] = runs_df.apply(
    lambda r: np.std([r[f"{c}/rank"] for c in linear_sets]),
    axis=1
) 

----- TEST_FEW_SHOT/FC100 ------


,name,test_few_shot/fc100,rank
9,moco-resnet50-imagenet100_mlp_2_16_proj-cat_crop_color,0.502925,0
6,moco-resnet50-imagenet100_mlp_3_16_proj-cat_crop_color,0.490419,1
5,moco-resnet50-imagenet100_augself_OFFICIAL,0.489706,2
4,moco-resnet50-imagenet100_baseline_OFFICIAL,0.457837,3
3,moco-resnet50-imagenet100_mlp_3_256_proj-cat_crop_color,0.000000,4
7,moco-resnet50-imagenet100_augself,0.000000,5
8,moco-resnet50-imagenet100_mlp_2_32_proj-cat_crop_color,0.000000,6
0,moco-resnet50-imagenet100_BASELINE_PAPER,NaN,7
1,moco-resnet50-imagenet100_AUGSELF_PAPER,NaN,8


----- TEST_LINEAR/FOOD101 ------


,name,test_linear/food101,rank
7,moco-resnet50-imagenet100_augself,0.622297,0
9,moco-resnet50-imagenet100_mlp_2_16_proj-cat_crop_color,0.612317,1
6,moco-resnet50-imagenet100_mlp_3_16_proj-cat_crop_color,0.611842,2
5,moco-resnet50-imagenet100_augself_OFFICIAL,0.610218,3
1,moco-resnet50-imagenet100_AUGSELF_PAPER,0.607800,4
0,moco-resnet50-imagenet100_BASELINE_PAPER,0.596700,5
4,moco-resnet50-imagenet100_baseline_OFFICIAL,0.595406,6


----- TEST_LINEAR_LOOC_LIKE_ACC/IMAGENET100 ------


...

,name,test_few-shot_5-way_5-shot/fc100,rank
7,moco-resnet50-imagenet100_augself,0.514888,0
9,moco-resnet50-imagenet100_mlp_2_16_proj-cat_crop_color,0.502925,1
6,moco-resnet50-imagenet100_mlp_3_16_proj-cat_crop_color,0.490419,2
5,moco-resnet50-imagenet100_augself_OFFICIAL,0.489706,3
1,moco-resnet50-imagenet100_AUGSELF_PAPER,0.487700,4
4,moco-resnet50-imagenet100_baseline_OFFICIAL,0.457837,5
0,moco-resnet50-imagenet100_BASELINE_PAPER,0.438800,6
3,moco-resnet50-imagenet100_mlp_3_256_proj-cat_crop_color,0.000000,7


----- TEST_FEW-SHOT_5-WAY_5-SHOT/CUB200 ------


,name,test_few-shot_5-way_5-shot/cub200,rank
7,moco-resnet50-imagenet100_augself,0.588081,0
1,moco-resnet50-imagenet100_AUGSELF_PAPER,0.573500,1
5,moco-resnet50-imagenet100_augself_OFFICIAL,0.571694,2
9,moco-resnet50-imagenet100_mlp_2_16_proj-cat_crop_color,0.570188,3
6,moco-resnet50-imagenet100_mlp_3_16_proj-cat_crop_color,0.569381,4
0,moco-resnet50-imagenet100_BASELINE_PAPER,0.569200,5
3,moco-resnet50-imagenet100_mlp_3_256_proj-cat_crop_color,0.000000,6


----- TEST_LINEAR/CARS ------


,name,test_linear/cars,rank
6,moco-resnet50-imagenet100_mlp_3_16_proj-cat_crop_color,0.394603,0
9,moco-resnet50-imagenet100_mlp_2_16_proj-cat_crop_color,0.387390,1
5,moco-resnet50-imagenet100_augself_OFFICIAL,0.376446,2
7,moco-resnet50-imagenet100_augself,0.376073,3
1,moco-resnet50-imagenet100_AUGSELF_PAPER,0.373500,4
0,moco-resnet50-imagenet100_BASELINE_PAPER,0.338600,5
4,moco-resnet50-imagenet100_baseline_OFFICIAL,0.338142,6


In [14]:
runs_df.sort_values("mean_rank")[["name", "rank_mean", "rank_std"]].head(20)

,name,rank_mean,rank_std
35,moco-resnet18-stl10_augself,0.454545,0.497930
1,moco-resnet18-stl10_AUGSELF,4.000000,5.984829
71,moco-resnet18-stl10_mlp_3_16_crop_color,6.090909,3.895325
72,moco-resnet18-stl10_mlp_2_64_crop_color,6.818182,6.938931
70,moco-resnet18-stl10_mlp_2_16_crop_color,7.181818,3.833303
34,moco-resnet18-stl10_baseline,8.545455,4.979296
74,moco-resnet18-stl10_mlp_2_32_crop_color,8.636364,5.596634
43,moco-resnet18-stl10_mlp_3_1024_proj-add_crop_color,12.363636,4.162008
59,moco-resnet18-stl10_mlp_2_128_proj-cat_crop_color,12.454545,6.065755
60,moco-resnet18-stl10_mlp_2_128_proj-add_crop_color,13.363636,5.612854
